In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [10]:
from typing import Dict
from acctrack.io.athena_raw_root import AthenaRawRootReader

import awkward as ak
import numpy as np

In [4]:
input_dir = "/media/DataOcean/data/ITk/BugFixedSamples"
reader = AthenaRawRootReader(input_dir)

Total number of files: 1 in /media/DataOcean/data/ITk/BugFixedSamples


In [5]:
tree = reader.read()

Number of entries in GNN4ITk: 200


In [6]:
particle_branch_prefix = "Part_"
# "vParentID", "vParentBarcode" are not used.
particle_columns = [
    "event_number", "barcode", 
    "px", "py", "pz", "pt", "eta", 
    "vx", "vy", "vz",
    "radius", "status", "charge", "pdg_id",
    "passed", "vProdNin", "vProdNout", "vProdStatus", 
    "vProdBarcode"
]

particle_branch_names = [particle_branch_prefix + col for col in particle_columns]

In [7]:
particles = tree.arrays(particle_branch_names, library="np")

In [9]:
type(particles)

dict

In [15]:
particles["Part_event_number"][0]

array([   0,    0,    0, ..., 3270, 3270, 3270], dtype=int32)

In [16]:
def create_arrays(branch_arrays: Dict[str, np.ndarray]):
    variables = list(branch_arrays.keys())
    num_evts = len(branch_arrays[variables[0]])
    all_arrays = []
    print("num_evts:", num_evts)
    for i in range(num_evts):
        inputs = [branch_arrays[x][i] for x in variables]
        array = np.stack(inputs, axis=1)
        all_arrays.append(array)

    return all_arrays

In [21]:
len(particles[list(particles.keys())[0]])

200

In [17]:
par_array = create_arrays(particles, 0)

In [18]:
par_array.shape

(185528, 19)

In [14]:
particle_pt = particles[particle_branch_prefix + "pt"]

In [15]:
type(particle_pt), len(particle_pt[0])

(awkward.highlevel.Array, 185528)

In [25]:
df = ak.to_dataframe(particle_pt[0])

In [26]:
df

,values
entry,
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,190916.156250
...,...
185523,368.097168
185524,359.283356
185525,371.639557


In [24]:
df.loc[(0, 0)]

values    0.0
Name: (0, 0), dtype: float32